In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
import seaborn as sns
import lightgbm as lgb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
from tqdm import tqdm
import random
import seaborn as sns
import math
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import xgboost as xgb
import gc
from lightgbm import LGBMRegressor
import optuna
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from optuna.integration import LightGBMPruningCallback


In [ ]:
sales=pd.read_pickle('sales.pkl')
#weekly_sales=pd.read_csv('data/processed/weekly_sales.csv')
#monthly_sales=pd.read_csv('data/processed/monthly_sales.csv')

calendar=pd.read_csv('data/raw/calendar.csv')
prices=pd.read_csv('data/raw/sell_prices.csv')

'''sales=sales.iloc[:,6:]


for i in range (len(sales)):   #colocando null nas primeiras vendas 0
    if sales.iloc[i,0] == 0:
        sales.iloc[i,0] = np.nan
    for k in range(1,1913):
        if sales.iloc[i,k] == 0 and str(sales.iloc[i,k-1])=='nan':
            sales.iloc[i,k] =np.nan
        else:
            break'''

"sales=sales.iloc[:,6:]\n\n\nfor i in range (len(sales)):   #colocando null nas primeiras vendas 0\n    if sales.iloc[i,0] == 0:\n        sales.iloc[i,0] = np.nan\n    for k in range(1,1913):\n        if sales.iloc[i,k] == 0 and str(sales.iloc[i,k-1])=='nan':\n            sales.iloc[i,k] =np.nan\n        else:\n            break"

In [ ]:
cluster=pd.read_csv('cluster1000d.csv').drop(columns='Unnamed: 0')
sales['cluster'] = cluster
sales.to_pickle('sales.pkl')

In [93]:
sales['mean']=sales.iloc[:,:-2].mean(axis=1)

In [94]:
sales.drop(columns=['id'], inplace=True)

KeyError: "['id'] not found in axis"

In [6]:
days=calendar['d'][-28:]
cluster_results = pd.DataFrame(columns = ['cluster','mean_cluster'] + list(days))


for x in list(cluster['cluster'].unique()):
    cluster_results.loc[x,'cluster']=x
    sales_mod=sales[sales['cluster']==x].drop(columns=['cluster','mean'])
    cluster_mean=pd.DataFrame(sales_mod.mean()).reset_index().rename(columns={'index':'d', 0:'sales'})

    df=cluster_mean.merge(calendar, how='right', on='d')

    df['roll2']= df["sales"].shift(1).rolling(window=2).mean().values
    df['roll3']= df["sales"].shift(1).rolling(window=3).mean().values
    df['roll5']= df["sales"].shift(1).rolling(window=5).mean().values
    df['roll7']= df["sales"].shift(1).rolling(window=7).mean().values
    df['roll30']= df["sales"].shift(1).rolling(window=30).mean().values
    df['roll10']= df["sales"].shift(1).rolling(window=10).mean().values

    df = df.drop(columns=['date','d','wm_yr_wk'])

    df = pd.get_dummies(df, columns=['weekday', 'wday', 'month', 'year', 'event_name_1',
       'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX',
       'snap_WI'])
    train=df.iloc[:1941,:]
    test=df.iloc[1941:,:].drop(columns='sales')

    X=train.dropna(subset=['sales']).drop(['sales'],axis=1)
    y=train['sales'].dropna()

    param_grid = {
        "n_estimators": [20,80,250,500],
        "learning_rate": [0.05,0.3],
        "num_leaves": [50,300,1000],
        "max_depth": [3,6,9],
    }
    model=LGBMRegressor()
    grid_search = GridSearchCV(model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

    grid_search.fit(X,y)

    best_model = grid_search.best_estimator_
    print('melhor RMSE:', np.sqrt(-grid_search.best_score_))
    cluster_results.loc[cluster_results['cluster']==x,'d_1942':] = best_model.predict(test)      #adding next 28 days predicted value for each cluster
    cluster_results.loc[cluster_results['cluster']==x,'mean_cluster'] = cluster_mean['sales'].mean()   #adding mean column for each cluster

melhor RMSE: 0.2694193156525923
melhor RMSE: 0.3322070680776024
melhor RMSE: 0.480029124796134
melhor RMSE: 0.42417047621734255
melhor RMSE: 0.3766095390275065
melhor RMSE: 0.2119537480038113
melhor RMSE: 0.49761902861602003
melhor RMSE: 0.28493775149096323
melhor RMSE: 0.2783870134185882
melhor RMSE: 0.34692831010553166
melhor RMSE: 0.3176338755616487
melhor RMSE: 0.29252465364813324
melhor RMSE: 0.9301736645044368
melhor RMSE: 0.2056902815626308
melhor RMSE: 0.1850824790003567
melhor RMSE: 0.3373771101087308
melhor RMSE: 0.9062974114094197
melhor RMSE: 0.7017744597998469
melhor RMSE: 0.29727343916678617
melhor RMSE: 0.4825445571824039
melhor RMSE: 0.5358022153509829
melhor RMSE: 0.3098265222965601
melhor RMSE: 0.21099118036185105
melhor RMSE: 0.3627099397318446
melhor RMSE: 0.3288377728858362
melhor RMSE: 0.3553075997652972
melhor RMSE: 0.2655750925538663
melhor RMSE: 0.23005520232889576
melhor RMSE: 0.4200522374609748
melhor RMSE: 0.3954154040172971
melhor RMSE: 0.4166661268211789
m

In [238]:
'''def plotImp(model, X , num = 20, fig_size = (40, 20)):
    feature_imp = pd.DataFrame({'Value':model.feature_importances_,'Feature':X.columns})
    plt.figure(figsize=fig_size)
    sns.set(font_scale = 5)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", 
                                                        ascending=False)[0:num])
    plt.title('LightGBM Features')
    plt.tight_layout()
    plt.savefig('lgbm_importances-01.png')
    plt.show()

plotImp(model, X_valid)'''

'def plotImp(model, X , num = 20, fig_size = (40, 20)):\n    feature_imp = pd.DataFrame({\'Value\':model.feature_importances_,\'Feature\':X.columns})\n    plt.figure(figsize=fig_size)\n    sns.set(font_scale = 5)\n    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", \n                                                        ascending=False)[0:num])\n    plt.title(\'LightGBM Features\')\n    plt.tight_layout()\n    plt.savefig(\'lgbm_importances-01.png\')\n    plt.show()\n\nplotImp(model, X_valid)'

In [95]:
cluster_results.to_csv('results500w.csv')

In [96]:
submission=pd.read_csv('data/raw/sample_submission.csv')

In [97]:
submission[['cluster','mean_prod']] = sales[['cluster','mean']].append(sales[['cluster','mean']]).reset_index(drop=True)

In [98]:
cluster_results.columns=list(cluster_results.columns[0:2])  +list( submission.columns[1:-2] )

In [99]:
submission.drop(columns=submission.columns[1:-2], inplace=True)

In [100]:
submission = submission.merge(cluster_results, on='cluster', how='left')

In [102]:
submission

,id,cluster,mean_prod,mean_cluster,F1,F2,F3,F4,F5,F6,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,460,0.608654,0.679195,0.683659,1.10789,1.07753,1.10789,1.13304,1.36829,...,1.13304,1.36829,1.20159,1.10509,1.10509,1.07472,1.10509,1.13304,1.36829,1.20159
1,HOBBIES_1_002_CA_1_validation,206,0.278087,1.19587,1.04046,1.45387,1.52859,1.478,1.58925,2.07834,...,1.58925,2.08446,1.63054,1.47152,1.45387,1.53699,1.478,1.58925,2.07834,1.62213
2,HOBBIES_1_003_CA_1_validation,420,0.369617,1.3057,1.27651,2.20762,2.03844,2.2034,2.24685,2.19002,...,2.26731,2.10205,2.54519,2.19896,2.13412,1.95629,2.2034,2.24685,2.19002,2.60899
3,HOBBIES_1_004_CA_1_validation,125,1.751706,1.00119,0.624875,2.58751,2.58751,2.58751,2.67152,2.82729,...,2.67152,2.79001,2.64977,2.55022,2.58751,2.55022,2.58751,2.67152,2.82729,2.68706
4,HOBBIES_1_005_CA_1_validation,441,1.032258,0.889018,0.83103,1.16978,1.25127,1.29526,1.30219,1.5709,...,1.314,1.58461,1.3164,1.25719,1.16978,1.26499,1.29526,1.30219,1.5709,1.30268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,478,0.536359,1.82813,1.62716,4.03555,4.02478,4.04005,4.05978,4.43627,...,4.08674,4.43357,4.38179,4.03767,4.05535,4.04952,4.05552,4.07277,4.43627,4.38449
60976,FOODS_3_824_WI_3_evaluation,478,0.376033,1.82813,1.62716,4.03555,4.02478,4.04005,4.05978,4.43627,...,4.08674,4.43357,4.38179,4.03767,4.05535,4.04952,4.05552,4.07277,4.43627,4.38449
60977,FOODS_3_825_WI_3_evaluation,478,0.893814,1.82813,1.62716,4.03555,4.02478,4.04005,4.05978,4.43627,...,4.08674,4.43357,4.38179,4.03767,4.05535,4.04952,4.05552,4.07277,4.43627,4.38449
60978,FOODS_3_826_WI_3_evaluation,478,0.737525,1.82813,1.62716,4.03555,4.02478,4.04005,4.05978,4.43627,...,4.08674,4.43357,4.38179,4.03767,4.05535,4.04952,4.05552,4.07277,4.43627,4.38449


In [103]:
submission['factor'] = submission['mean_prod']/submission['mean_cluster']

In [104]:
for col in submission.iloc[:,4:-1].columns:  
    submission.loc[:,col]*=submission['factor'] 

In [105]:
submission.drop(columns =['cluster','mean_prod','mean_cluster', 'factor'], inplace=True)

In [106]:
for col in range(len(sales.iloc[:,-30:-2].columns)):
    submission.iloc[:3490,col+1] = sales.iloc[:,-30+col]

In [107]:
submission.to_csv('submission_500w.csv', index=False)